In [1]:
import pandas as pd
from faker import Faker
from graphdatascience import GraphDataScience
import os
from dotenv import load_dotenv

/Users/abhisheknarisepallivenkatasai/SUPPLY_CHAIN_OPTIMIZATION/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USER")
pwd = os.getenv("NEO4J_PASSWORD")

In [4]:
gds = GraphDataScience(uri, auth=(user, pwd))

### CLEAR EXISTING DATA IN NEO4J 

In [6]:
gds.run_cypher('MATCH(N) DETACH DELETE N')
gds.run_cypher('CALL apoc.schema.assert({},{})')

,label,key,keys,unique,action
0,RECEPTION,shipmentId,[shipmentId],False,DROPPED
1,DEPARTURE,shipmentId,[shipmentId],False,DROPPED
2,TRANSPORT,shipmentId,[shipmentId],False,DROPPED
3,DELIVERY,shipmentId,[shipmentId],False,DROPPED
4,TRANSFER,shipmentId,[shipmentId],False,DROPPED
5,Destination,airportId,[airportId],True,DROPPED
6,DeparturePoint,airportId,[airportId],True,DROPPED
7,Airport,airportId,[airportId],True,DROPPED
8,TransferPoint,airportId,[airportId],True,DROPPED
9,DepartureWarehouse,airportId,[airportId],True,DROPPED


# Load the Cargo2000 dataset

In [7]:
df = pd.read_csv('https://s-cube-network.eu/c2k-files/c2k_data_comma.csv', dtype=str)

In [8]:
df

,nr,i1_legid,i1_rcs_p,i1_rcs_e,i1_dep_1_p,i1_dep_1_e,i1_dep_1_place,i1_rcf_1_p,i1_rcf_1_e,i1_rcf_1_place,...,o_dep_3_p,o_dep_3_e,o_dep_3_place,o_rcf_3_p,o_rcf_3_e,o_rcf_3_place,o_dlv_p,o_dlv_e,o_hops,legs
0,0,5182,199,218,210,215,609,935,736,256,...,?,?,?,?,?,?,780,434,1,2
1,1,6523,844,584,90,297,700,1935,1415,431,...,?,?,?,?,?,?,3870,445,1,2
2,2,5878,4380,4119,90,280,456,905,547,700,...,?,?,?,?,?,?,550,1520,1,1
3,3,1275,759,169,240,777,173,340,577,349,...,?,?,?,?,?,?,3780,159,1,1
4,4,8117,1597,1485,150,241,411,585,612,128,...,?,?,?,?,?,?,4140,4797,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3938,3939,4225,122,19,240,278,815,830,761,174,...,?,?,?,?,?,?,1665,1300,2,3
3939,3940,14017,2028,413,270,1825,605,2610,2535,349,...,?,?,?,?,?,?,3780,807,2,1
3940,3941,4660,1356,178,240,1359,815,760,716,609,...,?,?,?,?,?,?,5100,4381,2,1
3941,3942,6472,2692,1856,90,867,700,1060,1049,113,...,?,?,?,?,?,?,3780,945,2,2


In [9]:
LEGS = [1,2,3]
SEGMENTS = [1,2,3]
GOINGS = ['i','o']
SERVICES = ['rcs', 'dlv']


def get_last_i1_rcf_place(row):
    for s in [3,2,1]:
        if row[f'i1_rcf_{s}_place'] != '?':
            return row[f'i1_rcf_{s}_place']
    raise Exception("cannot find last rcf place")

def get_last_outbound_rcf_place(row):
    for s in [3,2,1]:
        if row[f'o_rcf_{s}_place'] != '?':
            return row[f'o_rcf_{s}_place']
    raise Exception("cannot find last rcf place")

In [10]:
df['last_o_rcf_place'] = df.apply(get_last_outbound_rcf_place, axis =1)
df['last_i_rcf_place'] = df.apply(get_last_i1_rcf_place, axis =1)

In [12]:
# Create Indexes
gds.run_cypher('CREATE CONSTRAINT airport_unique IF NOT EXISTS FOR (n:Airport) REQUIRE n.airportId IS UNIQUE')

gds.run_cypher('CREATE CONSTRAINT airport_entry_unique IF NOT EXISTS FOR (n:EntryPoint) REQUIRE n.airportId IS UNIQUE')

gds.run_cypher('CREATE INDEX check_in_shipment_id IF NOT EXISTS FOR ()-[r:RECEPTION]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_departure_checkpoint_unique IF NOT EXISTS FOR (n:DepartureWarehouse) REQUIRE n.airportId IS UNIQUE')

gds.run_cypher('CREATE INDEX confirm_shipment_id IF NOT EXISTS FOR ()-[r:DEPARTURE]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_departure_unique IF NOT EXISTS FOR (n:DeparturePoint) REQUIRE n.airportId IS UNIQUE')

gds.run_cypher('CREATE INDEX depart_shipment_id IF NOT EXISTS FOR ()-[r:TRANSPORT]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_arrival_unique IF NOT EXISTS FOR (n:ArrivalWarehouse) REQUIRE n.airportId IS UNIQUE')

gds.run_cypher('CREATE INDEX deliver_shipment_id IF NOT EXISTS FOR ()-[r:DELIVERY]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_destination_unique IF NOT EXISTS FOR (n:Destination) REQUIRE n.airportId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT airport_transfer_unique IF NOT EXISTS FOR (n:TransferPoint) REQUIRE n.airportId IS UNIQUE')

gds.run_cypher('CREATE INDEX transfer_shipment_id IF NOT EXISTS FOR ()-[r:TRANSFER]-() ON (r.shipmentId)')

""


In [13]:
# Load nodes and draw location at relationships
cols = df.columns.tolist()

airport_ids = set()
for col in cols:
    if 'place' in col:
        airport_ids.update([int(i) for i in df.loc[(df[col] != '?') & (df[col].notna()), col].unique().tolist()])

Faker.seed(0)
fake = Faker()
def single_name_city():
   n = fake.city()
   while ' ' in n: 
       n = fake.city()
   return n

airports = [{'id':airport_id , 'name':single_name_city()} for airport_id in airport_ids]

In [14]:
gds.run_cypher('''
    UNWIND $airports AS a
    WITH a.id AS airportId, a.name AS name
    MERGE(n0:Airport {airportId: airportId}) SET n0.name=name
    MERGE(n1:EntryPoint {airportId: airportId}) SET n1.name=name
    MERGE(n2:DepartureWarehouse{airportId: airportId}) SET n2.name=name
    MERGE(n3:DeparturePoint {airportId: airportId}) SET n3.name=name
    MERGE(n4:ArrivalWarehouse {airportId: airportId}) SET n4.name=name
    MERGE(n5:TransferPoint {airportId: airportId}) SET n5.name=name
    MERGE(n6:Destination {airportId: airportId}) SET n6.name=name

    MERGE(n0)<-[:LOCATED_AT]-(n1)
    MERGE(n0)<-[:LOCATED_AT]-(n2)
    MERGE(n0)<-[:LOCATED_AT]-(n3)
    MERGE(n0)<-[:LOCATED_AT]-(n4)
    MERGE(n0)<-[:LOCATED_AT]-(n5)
    MERGE(n0)<-[:LOCATED_AT]-(n6)
    RETURN count(n0), count(n1), count(n2), count(n3), count(n4), count(n5), count(n6)
    ''', params={'airports':airports})

,count(n0),count(n1),count(n2),count(n3),count(n4),count(n5),count(n6)
0,237,237,237,237,237,237,237


# CREATE RELATIONSHIPS 

### MAKING A hub-and-spoke MODEL

### 1. LOCATED_AT Relationships

    Airport ← LOCATED_AT ← [All Facility Types]

### 2. RECEPTION Relationships

    EntryPoint → RECEPTION → DepartureWarehouse (Inbound)

    TransferPoint → RECEPTION → DepartureWarehouse (Outbound)

### 3. DEPARTURE Relationships

    DepartureWarehouse → DEPARTURE → DeparturePoint (Segment 1) - INITIAL DEPARTURES

    ArrivalWarehouse → DEPARTURE → DeparturePoint (Segments 2-3) - INTERMEDIATE DEPARTURES

### 4. TRANSPORT Relationships

    DeparturePoint → TRANSPORT → ArrivalWarehouse

### 5. TRANSFER Relationships

    ArrivalWarehouse → TRANSFER → TransferPoint

### 6. DELIVERY Relationships

    TransferPoint → DELIVERY → Destination


In [15]:
# (n:EntryPoint)-[r:RECEPTION]-(m:DepartureWarehouse) RCS inbound
for l in LEGS:
    sub_dict = df.loc[(df[f'i{l}_legid'] != '?') & (df[f'i{l}_legid'].notna()),
                      ['nr', f'i{l}_legid', f'i{l}_rcs_p', f'i{l}_rcs_e', f'i{l}_dep_1_place']].to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH toInteger(relMap.nr) AS shipmentId,
            toInteger(relMap.i{l}_dep_1_place) AS airportId,
            toInteger(relMap.i{l}_legid) AS legId,
            toInteger(relMap.i{l}_rcs_e) AS effectiveMinutes,
            toInteger(relMap.i{l}_rcs_p) AS plannedMinutes
        MATCH(n1:EntryPoint {{airportId: airportId}})
        MATCH(n2:DepartureWarehouse {{airportId: airportId}})
        MERGE(n1)-[r:RECEPTION {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=plannedMinutes,
        r.effectiveMinutes=effectiveMinutes
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':l})
    print(res)

   count(r)
0      3942
   count(r)
0      2624
   count(r)
0      1366


In [16]:
# (n:TransferPoint)-[r:RECEPTION]-(m:DepartureWarehouse) RCS outbound

sub_dict = df.loc[(df[f'o_legid'] != '?') & (df[f'o_legid'].notna()),
                  ['nr', 'o_legid', 'o_rcs_p', 'o_rcs_e', 'o_dep_1_place']].to_dict('records')
gds.run_cypher('''
    UNWIND $relMaps AS relMap
    WITH toInteger(relMap.nr) AS shipmentId,
        toInteger(relMap.o_dep_1_place) AS airportId,
        toInteger(relMap.o_legid) AS legId,
        toInteger(relMap.o_rcs_e) AS effectiveMinutes,
        toInteger(relMap.o_rcs_p) AS plannedMinutes
    MATCH(n1:TransferPoint {airportId: airportId})
    MATCH(n2:DepartureWarehouse {airportId: airportId})
    MERGE(n1)-[r:RECEPTION {shipmentId: shipmentId, legId: legId, legNumber: -1}]->(n2)
    ON CREATE SET r.plannedMinutes=plannedMinutes,
        r.effectiveMinutes=effectiveMinutes
    RETURN count(r)
''', params={'relMaps':sub_dict})

,count(r)
0,3942


In [17]:
# (:DepartureWarehouse)-[r:DEPARTURE]-(:DeparturePoint) DEP1 inbound
for l in LEGS:
    print(f'== LEG {l} ======================')
    for s in SEGMENTS:
        print(f'-- SEGMENT {s} ----------------------')
        source_label = 'DepartureWarehouse'
        if s > 1:
            source_label = 'ArrivalWarehouse'
        sub_dict = df.loc[(df[f'i{l}_dep_{s}_place'] != '?') & (df[f'i{l}_legid'].notna()),
            ['nr', f'i{l}_legid', f'i{l}_dep_{s}_p', f'i{l}_dep_{s}_e', f'i{l}_dep_{s}_place']]\
            .to_dict('records')
        res = gds.run_cypher(f'''
            UNWIND $relMaps AS relMap
            WITH toInteger(relMap.nr) AS shipmentId,
                toInteger(relMap.i{l}_dep_{s}_place) AS airportId,
                toInteger(relMap.i{l}_legid) AS legId,
                toInteger(relMap.i{l}_dep_{s}_e) AS effectiveMinutes,
                toInteger(relMap.i{l}_dep_{s}_p) AS plannedMinutes
            MATCH(n1:{source_label} {{airportId: airportId}})
            MATCH(n2:DeparturePoint {{airportId: airportId}})
            MERGE(n1)-[r:DEPARTURE {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
            ON CREATE SET r.plannedMinutes=plannedMinutes,
                r.effectiveMinutes=effectiveMinutes
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l, 'segmentNumber':s})
        print(res)

== LEG 1 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1195
-- SEGMENT 3 ----------------------
   count(r)
0        23
== LEG 2 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      2624
-- SEGMENT 2 ----------------------
   count(r)
0       791
-- SEGMENT 3 ----------------------
   count(r)
0        14
== LEG 3 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      1366
-- SEGMENT 2 ----------------------
   count(r)
0       391
-- SEGMENT 3 ----------------------
   count(r)
0         8


In [18]:
# (:DepartureWarehouse)-[r:DEPARTURE]-(:DeparturePoint) DEP1 outbound
for s in SEGMENTS:
    source_label = 'DepartureWarehouse'
    if s > 1:
        source_label = 'ArrivalWarehouse'
    print(f'-- SEGMENT {s} ----------------------')
    sub_dict = df.loc[(df[f'o_dep_{s}_place'] != '?') & (df[f'o_legid'].notna()),
                      ['nr', f'o_legid', f'o_dep_{s}_p', f'o_dep_{s}_e', f'o_dep_{s}_place']]\
        .to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH toInteger(relMap.nr) AS shipmentId,
            toInteger(relMap.o_dep_{s}_place) AS airportId,
            toInteger(relMap.o_legid) AS legId,
            toInteger(relMap.o_dep_{s}_e) AS effectiveMinutes,
            toInteger(relMap.o_dep_{s}_p) AS plannedMinutes
        MATCH(n1:{source_label} {{airportId: airportId}})
        MATCH(n2:DeparturePoint {{airportId: airportId}})
        MERGE(n1)-[r:DEPARTURE {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=plannedMinutes,
            r.effectiveMinutes=effectiveMinutes
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1, 'segmentNumber':s})
    print(res)

-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1845
-- SEGMENT 3 ----------------------
   count(r)
0        26


In [19]:
# (:DeparturePoint)-[r:TRANSPORT]-(:ArrivalWarehouse) inbound
for l in LEGS:
    print(f'== LEG {l} ======================')
    for s in SEGMENTS:
        print(f'-- SEGMENT {s} ----------------------')
        sub_dict = df.loc[(df[f'i{l}_rcf_{s}_place'] != '?') & (df[f'i{l}_legid'].notna()),
                          ['nr', f'i{l}_legid', f'i{l}_rcf_{s}_p', f'i{l}_rcf_{s}_e', f'i{l}_dep_{s}_place', f'i{l}_rcf_{s}_place']]\
            .to_dict('records')
        res = gds.run_cypher(f'''
            UNWIND $relMaps AS relMap
            WITH toInteger(relMap.nr) AS shipmentId,
                toInteger(relMap.i{l}_dep_{s}_place) AS fromAirportId,
                toInteger(relMap.i{l}_rcf_{s}_place) AS toAirportId,
                toInteger(relMap.i{l}_legid) AS legId,
                toInteger(relMap.i{l}_rcf_{s}_e) AS effectiveMinutes,
                toInteger(relMap.i{l}_rcf_{s}_p) AS plannedMinutes
            MATCH(n1:DeparturePoint {{airportId: fromAirportId}})
            MATCH(n2:ArrivalWarehouse {{airportId: toAirportId}})
            MERGE(n1)-[r:TRANSPORT {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
            ON CREATE SET r.plannedMinutes=plannedMinutes,
                r.effectiveMinutes=effectiveMinutes
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l, 'segmentNumber':s})
        print(res)

== LEG 1 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1195
-- SEGMENT 3 ----------------------
   count(r)
0        23
== LEG 2 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      2624
-- SEGMENT 2 ----------------------
   count(r)
0       791
-- SEGMENT 3 ----------------------
   count(r)
0        14
== LEG 3 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      1366
-- SEGMENT 2 ----------------------
   count(r)
0       391
-- SEGMENT 3 ----------------------
   count(r)
0         8


In [20]:
# (:DeparturePoint)-[r:TRANSPORT]-(:ArrivalWarehouse) RCF outbound
for s in SEGMENTS:
    print(f'-- SEGMENT {s} ----------------------')
    sub_dict = df.loc[(df[f'o_rcf_{s}_place'] != '?') & (df[f'o_legid'].notna()),
                      ['nr', f'o_legid', f'o_rcf_{s}_p', f'o_rcf_{s}_e',  f'o_dep_{s}_place', f'o_rcf_{s}_place']]\
        .to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH toInteger(relMap.nr) AS shipmentId,
            toInteger(relMap.o_dep_{s}_place) AS fromAirportId,
            toInteger(relMap.o_rcf_{s}_place) AS toAirportId,
            toInteger(relMap.o_legid) AS legId,
            toInteger(relMap.o_rcf_{s}_e) AS effectiveMinutes,
            toInteger(relMap.o_rcf_{s}_p) AS plannedMinutes
        MATCH(n1:DeparturePoint {{airportId: fromAirportId}})
        MATCH(n2:ArrivalWarehouse {{airportId: toAirportId}})
        MERGE(n1)-[r:TRANSPORT {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=plannedMinutes,
            r.effectiveMinutes=effectiveMinutes
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1, 'segmentNumber':s})
    print(res)

-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1845
-- SEGMENT 3 ----------------------
   count(r)
0        26


In [21]:
# (:ArrivalWarehouse)-[r:DELIVERY]-(:TransferPoint) DLV inbound
for l in LEGS:
    sub_dict = df.loc[(df[f'i{l}_legid'] != '?') & (df[f'i{l}_legid'].notna()),
                      ['nr', f'i{l}_legid', f'i{l}_dlv_p', f'i{l}_dlv_e', 'last_i_rcf_place']].to_dict('records')
    res = gds.run_cypher(f'''
        UNWIND $relMaps AS relMap
        WITH toInteger(relMap.nr) AS shipmentId,
            toInteger(relMap.last_i_rcf_place) AS airportId,
            toInteger(relMap.i{l}_legid) AS legId,
            toInteger(relMap.i{l}_dlv_e) AS effectiveMinutes,
            toInteger(relMap.i{l}_dlv_p) AS plannedMinutes
        MATCH(n1:ArrivalWarehouse {{airportId: airportId}})
        MATCH(n2:TransferPoint {{airportId: airportId}})
        MERGE(n1)-[r:DELIVERY {{shipmentId: shipmentId, legId: legId, legNumber: $legNumber}}]->(n2)
        ON CREATE SET r.plannedMinutes=plannedMinutes,
            r.effectiveMinutes=effectiveMinutes
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':l})
    print(res)

   count(r)
0      3942
   count(r)
0      2624
   count(r)
0      1366


In [22]:
# (:ArrivalWarehouse)-[r:DELIVERY]-(:Destination) DLV outbound
sub_dict = df.loc[(df[f'o_legid'] != '?') & (df[f'o_legid'].notna()),
                  ['nr', 'o_legid', f'o_dlv_p', f'o_dlv_e', 'last_o_rcf_place']].to_dict('records')
res = gds.run_cypher('''
    UNWIND $relMaps AS relMap
    WITH toInteger(relMap.nr) AS shipmentId,
        toInteger(relMap.last_o_rcf_place) AS airportId,
        toInteger(relMap.o_legid) AS legId,
        toInteger(relMap.o_dlv_e) AS effectiveMinutes,
        toInteger(relMap.o_dlv_p) AS plannedMinutes
    MATCH(n1:ArrivalWarehouse {airportId: airportId})
    MATCH(n2:Destination {airportId: airportId})
    MERGE(n1)-[r:DELIVERY {shipmentId: shipmentId, legId: legId, legNumber: $legNumber}]->(n2)
    ON CREATE SET r.plannedMinutes=plannedMinutes,
        r.effectiveMinutes=effectiveMinutes
    RETURN count(r)
''', params={'relMaps':sub_dict, 'legNumber':-1})
print(res)

   count(r)
0      3942


In [23]:
gds.run_cypher('''
    MATCH(a1:Airport)<-[:LOCATED_AT]-(d1:DeparturePoint)-[r:TRANSPORT]->(d2:ArrivalWarehouse)-[:LOCATED_AT]->(a2:Airport)
    WITH a1, a2, count(r) AS flightCount
    MERGE (a1)-[s:SENDS_TO]->(a2)
    SET s.flightCount = flightCount
    RETURN count(s) AS relationshipCount
''')

,relationshipCount
0,1205
